In [1]:
import pandas as pd
import numpy as np
from data_qc import *

In [2]:
!pip install openpyxl

In [3]:
data = pd.read_csv("/Users/andrewhill/RstudioProjects/vax_data/CO_VaxbyRegion_20221121.csv",index_col=["measure_date","state","region_id","age_grp"])
data.head()

,,,,dose1,primaryseries,booster1,booster2
measure_date,state,region_id,age_grp,,,,
12/13/2020,CO,coe,0-17 years,NaN,NaN,NaN,NaN
12/14/2020,CO,coe,0-17 years,NaN,NaN,NaN,NaN
12/15/2020,CO,coe,0-17 years,NaN,NaN,NaN,NaN
12/16/2020,CO,coe,0-17 years,NaN,NaN,NaN,NaN
12/17/2020,CO,coe,0-17 years,0.0,0.0,0.0,NaN


In [4]:
def run_qc(data: pd.DataFrame, tests: List[GenericQC], filename="qc_report.xlsx") -> pd.DataFrame:
    reports = []
    for test in tests:
        # Instantiate the test
        qc = test()
        print(f"Running '{qc.name}'")
        # Run the test
        qc_result = qc.check(data)
        print(f"\tPass: {qc_result}")
        # Get the results
        reports.append(qc.report_df())
    
    report_df = pd.concat(reports)
    report_df.to_excel(filename)
    issues = (report_df.index.get_level_values("Pass") == False).sum()
    print(f"{issues} issues detected. Report written to '{filename}'")
    return report_df

In [5]:
# Both checks will fail, there are missing and negative values.
qc_report_df = run_qc(data,[NegativeValuesQC,MissingValuesQC])

Running 'NegativeValuesQC'
	Pass: False
Running 'MissingValuesQC'
	Pass: False
1984 issues detected. Report written to 'qc_report.xlsx'


In [6]:
# Now try manually removing these values
data_mod = data.copy()
data_mod.fillna(0,inplace=True)
data_mod[data_mod < 0] = 0

In [7]:
qc_report_df_good = run_qc(data_mod,[NegativeValuesQC,MissingValuesQC],filename="qc_report_good.xlsx")

Running 'NegativeValuesQC'
	Pass: True
Running 'MissingValuesQC'
	Pass: True
0 issues detected. Report written to 'qc_report_good.xlsx'
